In [2]:
import subprocess
import psycopg2
import re

In [3]:
# Connect to an existing database
executable_path = './main-pol-sta'
p = "3" # A prime 
with psycopg2.connect("dbname=p-rank-2") as conn:
    cur_2 = conn.cursor()
    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Query the database and obtain data as Python objects.
        cur.execute("""
                    WITH OrderedRecords AS (
                        SELECT
                            D,
                            L_pol,
                            L_Q2_relations,
                            ROW_NUMBER() OVER (ORDER BY D) AS RowNum
                        FROM p_3
                    )
                    SELECT D, L_pol
                    FROM OrderedRecords
                    WHERE RowNum >= (SELECT RowNum FROM OrderedRecords WHERE D = 134059) AND L_Q2_relations IS NULL
                    ORDER BY D;
                    """)
        records = cur.fetchall()
        
        for record in records:
            D=record[0]
            print("\nDisc: ", D, "\n")
            #print(record[1][1:-1])
            pol_vec = record[1][1:-1].split(', ')
            #print(pol_vec)
            relations = ['-', '-', '-', '-']
            index=0
            for pol in pol_vec:
                print("Pol: ", pol)
                program_arguments = [p, pol]
                # Construct the command by combining the executable path and arguments
                command = [executable_path]+program_arguments
                
                # Run the C program with arguments and capture its output
                try:
                    # Start the subprocess
                    with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as proc:
                        try:
                            # Wait for the command to complete, with a timeout
                            stdout, stderr = proc.communicate(timeout=300)
                            full_output = stdout
                            
                            first_quote_pos = full_output.find('"""')
                            second_quote_pos = full_output.find('"""', first_quote_pos + 3)
                            if first_quote_pos != -1 and second_quote_pos != -1:
                                output = full_output[first_quote_pos + 3:second_quote_pos]
                                modified_s = re.sub(r'(\d)(?![,\]])', r'\1*', output)
                                relations[index] = modified_s
                                print(output, "\n")
                                
                            else:
                                print("The C program crashed\n")

                                
                        except subprocess.TimeoutExpired:
                            print("The subprocess did not complete within 5 minutes and will be terminated.\n")
                            proc.terminate()  # Explicitly terminate the subprocess
                            proc.wait()  # Wait for the subprocess to terminate
                            # Handle the timeout situation, such as cleanup or logging
                            
                except Exception as e:
                    # Handle other exceptions
                    print(str(e))
                index+=1
            #if len(relations)==4:
            cur_2.execute("""
                            UPDATE p_3
                            SET L_Q2_relations=%s
                            WHERE D=%s;
                        """, (str(relations), D))
            conn.commit()
            print("Changes saved to db\n")
            # Make the changes to the database persistent
        


Disc:  647359 

Pol:  s^6 - 2*s^5 - 242*s^4 + 2349*s^3 + 174496*s^2 + 2333557*s + 11466553
The subprocess did not complete within 5 minutes and will be terminated.

Pol:  s^6 + 87*s^4 - 5131*s^3 + 163732*s^2 - 1841596*s + 10627784
The subprocess did not complete within 5 minutes and will be terminated.

Pol:  s^6 + 514*s^4 + 66049*s^2 + 2589436
[a_c^1, a^6a_b^1b_c^2, a^6a_b^2c^6] 

Pol:  s^6 - 2*s^5 - 352*s^4 - 2927*s^3 + 196272*s^2 - 68439*s + 3336959
The subprocess did not complete within 5 minutes and will be terminated.

Changes saved to db


Disc:  654468 

Pol:  s^6 - 252*s^4 + 15876*s^2 + 23560848
[a_b^2, a^6a_c^1b^6, a^3a_c^1b_c^1] 

Pol:  s^6 + 180*s^4 + 8100*s^2 + 23560848
The subprocess did not complete within 5 minutes and will be terminated.

Pol:  s^6 + 630*s^4 + 99225*s^2 + 5890212
The C program crashed

Pol:  s^6 - 2*s^5 + 43*s^4 + 2274*s^3 + 161742*s^2 + 2012040*s + 7231176
The C program crashed

Changes saved to db


Disc:  660356 

Pol:  s^6 - 506*s^4 + 64009*s^2 + 

KeyboardInterrupt: 